# Visualization with Bokeh

This problem involves building an interactive visualization. Bokeh is a Python interactive visualization library that targets modern web browsers for presentation. Using the [King County House Sales](http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data-original) data, I built a Bokeh visualization which allows the user explore how price varies with living room size and year built. I created a visualization that allows the user to toggle the X axis of a scatter plot between living room size and year built, with the y-axis always being price. I also added the hover tool so that if the user hovers over a datapoint in the living-room-size plot a window pops up that shows year built - and vice versa.

In [1]:
import pandas as pd
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Select, HoverTool, BoxSelectTool, LinearAxis
from bokeh.plotting import Figure, output_notebook, show

df = pd.read_csv("kc_house_data.csv")

output_notebook()

source = ColumnDataSource(data=dict(x=df['sqft_living'], y=df['price'], z=df['yr_built']))
source2 = ColumnDataSource(data=dict(x1=df['sqft_living'], x2=df['yr_built']))

plot = Figure(plot_width=400, plot_height=400, tools="hover")
plot.scatter('x', 'y', source=source)
plot.yaxis.axis_label = "Price"
# create new x-axis and hide old one
plot.xaxis.visible = False
xaxis = LinearAxis(axis_label="Living Room Size")
plot.add_layout(xaxis, 'below')

plot.select(dict(type=HoverTool)).tooltips = {"Other X Value":"@z"}
callback = CustomJS(args=dict(source=source, source2=source2, xaxis=xaxis), code="""
        var data = source.get('data');
        var data2 = source2.get('data');
        var f = cb_obj.get('value')
        if (f=="Year Built"){
            data['x'] = data2['x2']
            data['z'] = data2['x1']
            xaxis.attributes.axis_label = "Year Built";
            xaxis.trigger('change');
            
        }
        if (f=="Living Room Size"){
            data['x'] = data2['x1']
            data['z'] = data2['x2']
            xaxis.attributes.axis_label = "Living Room Size";
            xaxis.trigger('change');
        }
        source.trigger('change');
    """)

slider = Select(title="Choose X:", value="Living Room Size", options=["Living Room Size", "Year Built"], callback=callback)

layout = column(slider, plot)

show(layout)

Loading BokehJS ...